In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
import copy

import altair as alt

In [12]:
data = pd.read_parquet("../data/adaptation_rate_01.parquet", engine="pyarrow")


In [13]:
# rename columns
headers = {"0": "adaptation_rate", "1": "iteration"}
for gradient in range(10):
    headers[str(gradient + 2)] = f"g_{action}"


data = data.rename(columns=headers)
data.head()

,adaptation_rate,iteration,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9
0,10.0,0.0,0.577549,0.656321,0.271209,0.542442,0.277479,0.256974,0.092969,0.018171,-0.029474,-0.014134
1,10.0,1.0,0.298428,-0.387148,0.111329,-0.145484,0.134269,0.276029,-0.091504,-0.097858,-0.092527,-0.036473
2,10.0,2.0,0.661842,0.345827,-0.431179,0.074868,0.087065,0.008113,-0.037473,-0.057193,-0.104390,-0.021774
3,10.0,3.0,1.344562,-0.556679,0.000422,0.029916,-0.039735,-0.118825,-0.092347,-0.026853,-0.018151,-0.020484
4,10.0,4.0,0.319317,-0.041615,-0.056655,-0.028439,-0.135337,-0.117442,-0.050623,-0.034721,-0.042510,-0.018230


In [76]:
#color=alt.condition(
#        gradient_domain[0] <= alt.datum.a_0 <= gradient_domain[1],
#        alt.value("steelblue"),
#        alt.value("orange")
#)

array([1.e+01, 1.e+00, 1.e-01, 1.e-02, 1.e-03])

In [89]:
gradient_domain = (-1, 1)
action_names = list(data.columns)[2:]

base = alt.Chart().mark_line(clip=True).encode(
    x="iteration:O",
    #tooltip=["iteration", "adaptation_rate", "a_0"]
).properties(
    height=200,
    width=600
)

base += base.mark_point(clip=True).encode(
    x="iteration:O"
).properties(
    height=200,
    width=600
)


chart = alt.vconcat(data=data)
for adaptation_rate in data["adaptation_rate"].unique():
    row = alt.hconcat()
    for action in action_names:
        row |= base.encode(
            y=alt.Y(field=action,
                    type="quantitative",
                    scale=alt.Scale(domain=gradient_domain,
                                    zero=True
                                   ),
                    axis=alt.Axis(title=f"{action}, {adaptation_rate}")
               ),
            tooltip=["iteration", "adaptation_rate", action]
        ).transform_filter(
            f"datum.adaptation_rate == {adaptation_rate}"
        )
    chart &= row
#chart.mark_line() + chart.mark_circle()
chart.save("adaptation_rate.html")

['a_0', 'a_1', 'a_2', 'a_3', 'a_4', 'a_5', 'a_6', 'a_7', 'a_8', 'a_9']